
# Projeto 2 - SCC0252 (Visualização Computacional)


*   Otávio Ferracioli Coletti - 11767796
*   Fernando César Lopes Barbosa Filho - 10260559

\\

---

In [90]:
# Bibliotecas importadas
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import Button, Layout
from matplotlib.spines import rcParams
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib import legend
from pandas import Series
import ipywidgets
import random
import plotly
import io

In [91]:
# Conjunto de dados do Spotify
url = 'https://raw.githubusercontent.com/otaviofcoletti/VIZCOMP/main/SpotifyFeatures(1).csv'
df = pd.read_csv(url)

## Heatmap

Geramos um *heatmap* que exibe os valores de correlação entre os atributos de todas as músicas do Spotify.


In [92]:
popular_music_rank = df.sort_values('popularity', ascending=False) 

In [93]:
def heatmap_interactive(**ch_list):

  categories = ['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness']
  features = []

  for i in categories:
    if ch_list[i]:
      features.append(i)

  selection = df[features]
  plt.figure(figsize=(15,10))
  sns.heatmap(selection.corr(),annot=True,cmap="coolwarm")
  plt.title('Heatmap of Correlations')
  plt.show()

In [94]:
cb_list = {description: widgets.Checkbox(
    value=random.choice([True, False]),
    description=description,
    disabled=False,
    indent=False
) for description in ['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness']}

Selecione nos **checkboxes** as *features* nas quais se deseja comparar a correlação no **heatmap**:

In [95]:
widgets.interactive(heatmap_interactive, **cb_list)

interactive(children=(Checkbox(value=False, description='valence', indent=False), Checkbox(value=True, descrip…

## Gráfico de Radar

Nesse tipo de visualização o usuário poderá selecionar quantas músicas se deseja analisar entre as mais populares, quais as *features* a serem os eixos e a respectiva ordem entre elas.

In [96]:
cinza = "#808080"
transparente = "rgba(0,0,0,0)"

def radar_chart_2(number_of_music=1,**dd_list):

  top5 = popular_music_rank[:number_of_music]

  lista_features = ['Feature 1','Feature 2','Feature 3','Feature 4','Feature 5','Feature 6','Feature 7','Feature 8']
  categories = ['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness']
  top5 = top5[['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness','track_name','artist_name']]
  fig = go.Figure()
  features = []

  for i in lista_features:

    if dd_list[i] != "Nenhum":
      features.append(dd_list[i])
  

  for i in range(0,number_of_music):

    fig.add_trace(go.Scatterpolar(
          r=list(top5.iloc[i]),
          theta=features,
          fill='toself',
          name= top5.iloc[i]['track_name'],
          legendgrouptitle_text="Músicas mais populares"
    ))


  fig.update_layout(
        font_color=cinza,
        font_size=15,
        autosize=True,
        height=400,
        margin=dict(l=40, r=40, t=25, b=25),
        paper_bgcolor=transparente,
        polar=dict(
            bgcolor=transparente,
            radialaxis=dict(
                range=[0, 1],
                linecolor=cinza,
                gridcolor=cinza,
                tickcolor=cinza,
            ),
            angularaxis=dict(
                rotation=120,
                direction="clockwise",
                linecolor=cinza,
                tickcolor=cinza,
                gridcolor=cinza,
            ),
        ),
    )
  
  fig.show()

In [97]:
lista_features = ['Feature 1','Feature 2','Feature 3','Feature 4','Feature 5','Feature 6','Feature 7','Feature 8']

In [98]:
charac=['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness','Nenhum']

In [99]:
index_list = [0,1,2,3,4,5,6,7,8]
random.seed(117)

dd_radar = {descri: widgets.Dropdown(
    options=charac,
    value=charac[index],
    description=descri,
    disabled=False,
) for index,descri in zip(index_list,lista_features) }

Através do gráfico de radar interativo podemos escolher quais as *features* musicais de interesse na análise, bem como escolher quantas e quais músicas comparar clicando na legenda associada. O usuário poderá escolher até 8 features, mas recomendamos entre 5 e 6 para facilitar a visualização.

In [115]:
ipywidgets.interactive(radar_chart_2, number_of_music=ipywidgets.IntSlider(min=1,max=15,step=1,value=5),**dd_radar)

interactive(children=(IntSlider(value=5, description='number_of_music', max=15, min=1), Dropdown(description='…

Podemos ver que, das músicas mais populares, todas possuem um *loudness* alto e uma *danceability* significativamente alta também. Notamos que, por exemplo, a acústica dessas músicas dificilmente ultrapassa 5. \\

Portanto, para ter sucesso é preciso fazer barulho, fazer o pessoal dançar e usar bastante da eletrônica na sua música! \\


## Treemap

Esse recurso nos permite visualizar a hierarquia de popularidade entre as diferentes músicas e artistas.

In [101]:
def treemap_plot(artist_or_genre, size_selection=10):

  if(artist_or_genre == 'Hierarquia por Artista'):
    fig = px.treemap(popular_music_rank[0:size_selection], path=['artist_name','track_name'],color='popularity', values ='popularity',title="Hierarquia das popularidades entre artistas",hover_name='artist_name',width=900, height=700)
  else:
    fig = px.treemap(popular_music_rank[0:size_selection], path=['genre', 'artist_name'], values ='popularity',title="Participação de cada artista na popularidade de cada gênero musical",hover_name='track_name',width=900, height=700)
    #fig = px.treemap(popular_music_rank[0:size_selection], path=['genre'], values ='popularity',title="Hierarquia das popularidades entre artistas",hover_name='genre',width=900, height=700)

  fig.update_traces(root_color="lightgrey")
  fig.update_traces(legendgrouptitle_text="Músicas mais populares")
  fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
  fig.update_layout(
    font_family="Helvetica",
    font_color="black",
  )
  fig.update_xaxes(title_font_family="Helvetica")
  fig.show()

In [102]:
ipywidgets.interact(treemap_plot, artist_or_genre=widgets.Dropdown(options=['Hierarquia por Artista','Hierarquia por Gênero Musical'],description='Selecione:'), size_selection=ipywidgets.IntSlider(min=5, max=100,step=5,value=25,description="N° músicas"))

interactive(children=(Dropdown(description='Selecione:', options=('Hierarquia por Artista', 'Hierarquia por Gê…

<function __main__.treemap_plot(artist_or_genre, size_selection=10)>

## Pré-processamento

Para não selecionar artistas com apenas um single de sucesso (conhecidos como 'one hit'), trabalharemos com artistas que possuem 10 ou mais músicas populares para se ter como base artistas de sucesso mais consistentes.

In [103]:
vc = df['artist_name'].value_counts()  # contando quantas vezes cada artista aparece
df = df[df['artist_name'].isin(vc.index[vc.gt(10)])]  # selecionando apenas aqueles que aparecem 10 vezes ou mais

artist_count = df.groupby('artist_name')[['popularity']].count().sort_values(by='popularity', ascending=True)
artist_sum = df.groupby('artist_name')[['popularity']].sum().sort_values(by='popularity', ascending=True)

artist_rank = artist_sum.popularity / artist_count.popularity
artist_rank = artist_rank.sort_values(ascending=False)
artist_rank = pd.DataFrame(artist_rank).reset_index()

print(artist_rank[:50])

               artist_name  popularity
0            Billie Eilish    0.805263
1               Juice WRLD    0.751111
2                   Khalid    0.750000
3              Post Malone    0.747692
4                   Offset    0.745000
5                 Coldplay    0.737692
6                  6ix9ine    0.729231
7                Bad Bunny    0.728387
8                 Lil Peep    0.725789
9             Metro Boomin    0.725714
10            XXXTENTACION    0.725192
11              Ed Sheeran    0.721852
12                    LANY    0.720000
13        Hailee Steinfeld    0.719091
14           Alec Benjamin    0.712308
15           Ariana Grande    0.708219
16                    Kygo    0.707778
17            Travis Scott    0.700208
18              Marshmello    0.696875
19         Michael Jackson    0.695714
20               21 Savage    0.691622
21          Camila Cabello    0.688235
22               Sam Smith    0.685882
23           Manuel Turizo    0.684545
24       Twenty One Pilot

Abaixo vamos selecionar apenas as músicas dos artistas acima e conhecer quais são suas características mais importantes.

In [104]:
top10_artistas = list(artist_rank['artist_name'][0:50])
top10 = pd.DataFrame()

for i in top10_artistas:
  top10 = top10.append(df[df['artist_name'].str.match(i)],ignore_index=True)

As funções abaixo irão nos ajudar a plotar os gráficos que indicarão como são em média as músicas dos artistas mais populares do Spotify.

In [105]:
def artist_rank_calculator(feature : str):

  artist_count = top10.groupby('artist_name')[[feature]].count().sort_values(by=feature, ascending=True)
  artist_sum = top10.groupby('artist_name')[[feature]].sum().sort_values(by=feature, ascending=True)
  artist_rank = artist_sum[feature] / artist_count[feature]
  artist_rank = artist_rank.sort_values(ascending=False)
  artist_rank = pd.DataFrame(artist_rank).reset_index()

  return artist_rank

In [106]:
def plot_artist_rank (number_of_artist=1,characterists='valence'):

  ar = artist_rank_calculator(characterists)
  feature = characterists
  title = characterists
  fig = px.bar(ar[0:number_of_artist].sort_values(by=feature, ascending=True), y="artist_name", x=feature, color=feature, title="Artistas mais populares e seu " + title + ' médio',orientation='h',width=900, height=700)
  fig.update_layout(xaxis_range=[0,1])

  fig.update_layout(
      font_family="Helvetica",
      font_color="black",
      title_font_family="Helvetica",
      title_font_color="black",
      legend_title_font_color="black"
  )

  fig.update_xaxes(title_font_family="Helvetica")
  fig.show()

In [107]:
lista_features = ['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness']

In [108]:
dd = widgets.Dropdown(
    options=lista_features,
    value='valence',
    description='Atributo da Música',
    disabled=False,
    layout=Layout(width='30%', height='big')
)

## Gráfico de barras


In [109]:
#ipywidgets.interactive(plot_artist_rank, number_of_artist=ipywidgets.IntSlider(min=1,max=50,step=1,value=10,description='N° artistas'), characterists=dd)

interactive(children=(IntSlider(value=10, description='N° artistas', max=50, min=1), Dropdown(description='Atr…

## Radviz

Temos aqui quatro eixos nos quais as músicas estarão dispostas para análise --- *energy, acousticness, danceability* e *popularity.*

In [110]:
def radviz_vs (genero_1 : str,genero_2 : str,genero_3 : str, number_of_music=150):
  
  fig = plt.figure( )
  plt.figure(figsize=(15,10))
  atributes = df[['acousticness','energy','danceability','popularity', 'genre']]
  pd.plotting.radviz(atributes[atributes['genre'].isin([genero_1])][:number_of_music], 'genre',color=['blue'])
  pd.plotting.radviz(atributes[atributes['genre'].isin([genero_2])][:number_of_music], 'genre',color=['red'])
  pd.plotting.radviz(atributes[atributes['genre'].isin([genero_3])][:number_of_music], 'genre',color=['green'])
  plt.title("Disposição das músicas dos gêneros ")
  plt.legend( loc='upper right', fontsize='larger' )

In [111]:
lista_generos = ['Genero 1','Genero 2','Genero 3']

In [112]:
genres=['Dance',
 'Rap',
 'Hip-Hop',
 'Pop',
 'Reggaeton',
 "Children's Music",
 'Folk',
 'Rock',
 'Indie',
 'Electronic',
 'Alternative',
 'Country',
 'Soul',
 'Blues',
 'Jazz',
 'Reggae',
 'R&B',
 'World',
 'Soundtrack',
 'Movie',
 'Classical',
 'Ska',
 'Anime',
 'Opera',
 'Comedy',
 'A Capella']

In [113]:
random.seed(10)
dd = {description: widgets.Dropdown(
    options=genres,
    value=genres[random.randint(0,len(genres)-1)],
    description=description,
    disabled=False,
) for description in lista_generos }

Escolha o gênero musical e número de músicas a comparar:

In [114]:
#ipywidgets.interact(radviz_vs, genero_1=dd['Genero 1'] ,genero_2=dd['Genero 2'] ,genero_3=dd['Genero 3'],number_of_music=ipywidgets.IntSlider(min=0, max=1000,step=25,value=150,description="N° músicas"))

interactive(children=(Dropdown(description='Genero 1', index=18, options=('Dance', 'Rap', 'Hip-Hop', 'Pop', 'R…

<function __main__.radviz_vs(genero_1: str, genero_2: str, genero_3: str, number_of_music=150)>